In [1]:
# coding: utf-8

# importing the required libraries
from mlbox.preprocessing import *
from mlbox.optimisation import *
from mlbox.prediction import *
import numpy as np

Using TensorFlow backend.
/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/kachunfung

In [2]:
# Preprocessing
df=Reader(sep=",").train_test_split(['train.csv','test.csv'],'Item_Outlet_Sales')


reading csv : train.csv ...
cleaning data ...
CPU time: 11.331454038619995 seconds

reading csv : test.csv ...
cleaning data ...
CPU time: 0.08772897720336914 seconds

> Number of common features : 11

gathering and crunching for train and test datasets ...
reindexing for train and test datasets ...
dropping training duplicates ...
dropping constant variables on training set ...

> Number of categorical features: 7
> Number of numerical features: 4
> Number of training samples : 8523
> Number of test samples : 5681

> Top sparse features (% missing values on train set):
Outlet_Size    28.3
Item_Weight    17.2
dtype: float64

> Task : regression
count     8523.000000
mean      2181.288914
std       1706.499616
min         33.290000
25%        834.247400
50%       1794.331000
75%       3101.296400
max      13086.964800
Name: Item_Outlet_Sales, dtype: float64


In [3]:
# removing the drift variables
df=Drift_thresholder().fit_transform(df)


computing drifts ...
CPU time: 0.47669386863708496 seconds

> Top 10 drifts

('Outlet_Establishment_Year', 0.025579140657205235)
('Outlet_Size', 0.023755435351466847)
('Item_Type', 0.014539550013354052)
('Item_Visibility', 0.012524094532208707)
('Outlet_Identifier', 0.011980699522146976)
('Outlet_Location_Type', 0.011418251706623028)
('Item_Fat_Content', 0.010253174769835116)
('Outlet_Type', 0.00944879497294182)
('Item_Weight', 0.008759344468078911)
('Item_MRP', 0.006550666615760425)

> Deleted variables : []
> Drift coefficients dumped into directory : save


In [7]:
df['train'].head()

,Item_Fat_Content,Item_Identifier,Item_MRP,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,Outlet_Size,Outlet_Type
0,Low Fat,FDA15,249.8092,Dairy,0.016047,9.30,1999.0,OUT049,Tier 1,Medium,Supermarket Type1
1,Regular,DRC01,48.2692,Soft Drinks,0.019278,5.92,2009.0,OUT018,Tier 3,Medium,Supermarket Type2
2,Low Fat,FDN15,141.6180,Meat,0.016760,17.50,1999.0,OUT049,Tier 1,Medium,Supermarket Type1
3,Regular,FDX07,182.0950,Fruits and Vegetables,0.000000,19.20,1998.0,OUT010,Tier 3,NaN,Grocery Store
4,Low Fat,NCD19,53.8614,Household,0.000000,8.93,1987.0,OUT013,Tier 3,High,Supermarket Type1


In [14]:
# setting the hyperparameter space
space={'ne__numerical_strategy':{"search":"choice","space":['mean','median']}, #missing cont.
'ne__categorical_strategy':{"search":"choice","space":["None"]}, #missing cat.
'ce__strategy':{"search":"choice","space":['label_encoding','entity_embedding','random_projection''dummification']},#categorical
'fs__strategy':{"search":"choice","space":['l1','variance','rf_feature_importance']}, #feature selection
'fs__threshold':{"search":"uniform","space":[0.01, 0.3]}, #% to be discarded
'est__max_depth':{"search":"choice","space":[3,5,7,9]},
'est__n_estimators':{"search":"choice","space":[250,500,700,1000]}}

In [15]:
# calculating the best hyper-parameter
best=Optimiser(scoring="mean_squared_error",n_folds=5).optimise(space,df,40)

##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 'mean', 'categorical_strategy': 'None'}
>>> CA ENCODER :{'strategy': 'random_projection'}   
>>> FEATURE SELECTOR :{'strategy': 'variance', 'threshold': 0.23145872333523304}
>>> ESTIMATOR :{'strategy': 'LightGBM', 'max_depth': 9, 'n_estimators': 1000, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.05, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': True, 'subsample': 0.9, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'nthread': -1, 'seed': 0}
  0%|          | 0/40 [00:00<?, ?it/s, best loss: ?]

/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-packages/mlbox/optimisation/optimiser.py:74: UserWarning: Optimiser will save all your fitted models into directory 'save/joblib'. Please clear it regularly.
  +str(self.to_path)+"/joblib'. Please clear it regularly.")
/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-packages/mlbox/optimisation/optimiser.py:296: UserWarning: Unknown or invalid scoring metric. neg_mean_squared_error is used instead.
  warnings.warn("Unknown or invalid scoring metric. "

/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)

/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecate

MEAN SCORE : neg_mean_squared_error = -1376609.0489763657
VARIANCE : 72200.68091140126 (fold 1 = -1416283.7221075075, fold 2 = -1351987.9455551938, fold 3 = -1399977.345603274, fold 4 = -1251096.1496808522, fold 5 = -1463700.0819350006)
CPU time: 7.729157209396362 seconds                 
##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 'median', 'categorical_strategy': 'None'}
>>> CA ENCODER :{'strategy': 'random_projection'}                            
>>> FEATURE SELECTOR :{'strategy': 'rf_feature_importance', 'threshold': 0.22568561152458538}
>>> ESTIMATOR :{'strategy': 'LightGBM', 'max_depth': 5, 'n_estimators': 500, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.05, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_jobs': -1, 'num_leaves': 31, 'objective': N

/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)

/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)

/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)

/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-

MEAN SCORE : neg_mean_squared_error = -1247563.0546571997                    
VARIANCE : 79940.21925410118 (fold 1 = -1301059.8023134277, fold 2 = -1269288.200427634, fold 3 = -1277372.011075569, fold 4 = -1089657.8870860487, fold 5 = -1300437.3723833186)
CPU time: 21.09471321105957 seconds                                          
##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 'median', 'categorical_strategy': 'None'}
>>> CA ENCODER :{'strategy': 'label_encoding'}                               
>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.010754493762353094}  
>>> ESTIMATOR :{'strategy': 'LightGBM', 'max_depth': 7, 'n_estimators': 250, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.05, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_jobs': -1, 'num

...

MEAN SCORE : neg_mean_squared_error = -1291668.1152057345                    
VARIANCE : 76067.40794347189 (fold 1 = -1330474.136948252, fold 2 = -1271270.0170415202, fold 3 = -1336854.0943675446, fold 4 = -1152760.4779114337, fold 5 = -1366981.849759922)
CPU time: 3.7738518714904785 seconds                                         
##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 'mean', 'categorical_strategy': 'None'}
>>> CA ENCODER :{'strategy': 'random_projection'}                            
>>> FEATURE SELECTOR :{'strategy': 'variance', 'threshold': 0.13507975449267742}
>>> ESTIMATOR :{'strategy': 'LightGBM', 'max_depth': 9, 'n_estimators': 250, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.05, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_jobs': -1, 'nu

/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)

/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)

/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)

/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-

MEAN SCORE : neg_mean_squared_error = -1229781.9784233102                    
VARIANCE : 77433.28402930892 (fold 1 = -1258867.566558055, fold 2 = -1231621.2206404735, fold 3 = -1278695.8523185088, fold 4 = -1081348.7176949685, fold 5 = -1298376.534904546)
CPU time: 2.4948551654815674 seconds                                         
##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 'median', 'categorical_strategy': 'None'}
>>> CA ENCODER :{'strategy': 'random_projection'}                            
>>> FEATURE SELECTOR :{'strategy': 'variance', 'threshold': 0.06288611488553003}
>>> ESTIMATOR :{'strategy': 'LightGBM', 'max_depth': 3, 'n_estimators': 700, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.05, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_jobs': -1, '

/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)

/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)

/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)

/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-

MEAN SCORE : neg_mean_squared_error = -1198463.8112615086                    
VARIANCE : 79720.6652517261 (fold 1 = -1237245.9471897504, fold 2 = -1217669.948648755, fold 3 = -1224971.700124739, fold 4 = -1043009.0084703376, fold 5 = -1269422.4518739611)
CPU time: 2.8391003608703613 seconds                                         
##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 'mean', 'categorical_strategy': 'None'}
>>> CA ENCODER :{'strategy': 'entity_embedding'}                             
>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.14900986623449436}   
>>> ESTIMATOR :{'strategy': 'LightGBM', 'max_depth': 5, 'n_estimators': 500, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.05, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_jobs': -1, 'num_le

In [ ]:
# predicting on the test dataset
Predictor().fit_predict(best,df)


fitting the pipeline ...


/Users/kachunfung/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


CPU time: 0.3914530277252197 seconds


In [8]:
target_name = 'Item_Outlet_Sales'

#submit = pd.read_csv("../input/sample_submission.csv",sep=',')
preds = pd.read_csv("save/"+target_name+"_predictions.csv")

#submit[target_name] =  preds[target_name+"_predicted"].values
#submit.to_csv("mlbox.csv", index=False)

In [9]:
preds

,Unnamed: 0,Item_Outlet_Sales_predicted
0,0,1771.853596
1,1,1368.225422
2,2,726.899395
3,3,2468.278730
4,4,6582.107177
5,5,1922.230351
6,6,345.428478
7,7,2322.046040
8,8,1390.479241
9,9,2823.982511
